# Preprocessing based on original annotation file - not very accurate

In [1]:
import sys
from os import listdir, makedirs
from os.path import join, exists

import numpy as np
import torch
import spacy
import codecs
import xmltodict
import pickle as pkl
from tqdm import tqdm
from allennlp.modules.elmo import Elmo, batch_to_ids

In [2]:
! python -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg')

prefix = "/home/texuanw/tools/allenNLP/" # or = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/"
options_file = prefix + "elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = prefix + "elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"
elmo = Elmo(options_file, weight_file, 2, dropout=0)

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [3]:
WORK_DIR = "../Data/bioc_FH_training/"
PROCESSED_DIR = "../Data/processed/"
OBJECTS_DIR = '../objects/'

torch.set_num_threads(8)

In [4]:
def load_pkl(path, file_name):
    with open(join(path, file_name), 'rb') as pkl_f:
        content = pkl.load(pkl_f)
    return content

In [5]:
def get_token_ent_type(token, entities):
    # entity: [start, end, ent_type, content]
    for entity in entities:
        if token.idx == entity[0]:
            return 'B-' + entity[2]
        elif token.idx > entity[0] and token.idx < entity[1]:
            return 'I-' + entity[2]
    return 'O'

In [6]:
def get_doc_labels(doc, entities, count):
    """
    Get labels of all tokens in a document.
    Each token is labeled as [entity_type, POS_type, dependency]
    """
    
    doc_label = []
    
    for sent in doc.sents:
        sent_label = []

        for idx, token in enumerate(sent):
            ent_type = get_token_ent_type(token, entities)

            if ent_type.startswith('B'):
                print(token, ent_type)
                count += 1

            token.ent_type_ = ent_type

            if not token.ent_type_ in entd.keys():
                entd[token.ent_type_] = len(entd)

            if not token.pos_ in posd.keys():
                posd[token.pos_] = len(posd)

            if not token.dep_ in depd.keys():
                depd[token.dep_] = len(depd)

            sent_label.append([entd[token.ent_type_],posd[token.pos_],depd[token.dep_]])

        doc_label.append(sent_label)
        
    return doc_label, count

In [ ]:
corpus = []
labels = []

entd = dict()
posd = dict()
depd = dict()
entd['<PAD>'] = len(entd)
posd['<PAD>'] = len(posd)
depd['<PAD>'] = len(depd)

texts = load_pkl(OBJECTS_DIR, 'texts.pkl')

count = 0
for title, doc in texts.items():
    entities = load_pkl(OBJECTS_DIR, 'entities_corrected.pkl')
    # print(entities[title]) ###
    
    doc = nlp(doc)
    doc_labels, count = get_doc_labels(doc, entities[title], count)
    labels.append(doc_labels)

    # sents to a list of lists of tokens
    doc_sent = list(map(lambda sent: list(map(lambda x: x.text, sent)), doc.sents))
    corpus.append(doc_sent)
    doc_emb = elmo(batch_to_ids(doc_sent)) # generate embemdding !! - list of lists [[embd_of_word, ], ]
    assert doc_emb['mask'].sum().item() == len(doc)
    
    with open(join(PROCESSED_DIR, title + 'pkl'), 'wb') as f:
        pkl.dump(doc_emb, f)

In [8]:
print(count)

1780


In [9]:
print('labels:', labels[0][0])
print('\ncorpus:', corpus[0][0])

labels: [[1, 1, 1], [1, 2, 2], [1, 2, 2], [1, 3, 3], [1, 3, 4], [1, 4, 5], [1, 4, 6], [1, 5, 7], [1, 1, 1], [1, 3, 8], [1, 5, 7], [1, 1, 1], [1, 3, 8], [1, 3, 9], [1, 6, 10], [1, 7, 11]]

corpus: ['A', 'detailed', 'comprehensive', 'family', 'history', 'was', 'obtained', 'from', 'the', 'patient', 'during', 'the', 'visit', 'today', '.', ' ']


In [10]:
np.save(join(OBJECTS_DIR, 'labeled_corpus'), np.array(labels))

with open(join(OBJECTS_DIR, 'label_dict.pkl'),'wb') as f:
    pkl.dump([entd, posd, depd], f)

In [11]:
# [doc][sent][token]
doc_emb['elmo_representations'][0][0][0].size()

torch.Size([1024])

In [12]:
print(len(entd))
print(entd)
# print(posd)
# print(depd)

6
{'<PAD>': 0, 'O': 1, 'B-FamilyMember': 2, 'B-Observation': 3, 'I-Observation': 4, 'I-FamilyMember': 5}
